# Model registry: MLFlow

Example on how to use the MLFlow model during the training of a model

## 1. Install MLFlow

In [2]:
 !pip install -r requeriments.txt
!pip freeze | grep mlflow

"grep" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



## 2. Start the MLFlow server.

To do this, run the following command in a terminal:

mlflow ui --host 0.0.0.0 --port 5001

## 3. Create a new experiment

In [3]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("mlflow-model-training-iris")

2025/03/05 18:02:40 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-model-training-iris' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/301193664384281066', creation_time=1741194160027, experiment_id='301193664384281066', last_update_time=1741194160027, lifecycle_stage='active', name='mlflow-model-training-iris', tags={}>

## 4. Log a model experiment result

In [4]:
# Lets first create a simple function to train a model
# The function will receive a set of hyperparameters and return the model and the accuracy
import datetime
from mlflow.models import infer_signature

def get_data():
    from sklearn import datasets

    # X, y = datasets.make_classification(n_samples=1000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, n_clusters_per_class=1, class_sep=0.5, random_state=40)
    # use iris dataset
    X, y = datasets.load_iris(return_X_y=True)
    return X, y

def train_model(hyperparameters):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import mlflow.sklearn
    X, y = get_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
    model = RandomForestClassifier(**hyperparameters)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    signature = infer_signature(X, model.predict(X))
    accuracy = accuracy_score(y_pred, y_test)
    return model, signature, accuracy

# Now lets train a model with some hyperparameters
hyperparameters = {"n_estimators": 10, "max_depth": 5}
model, signature, accuracy = train_model(hyperparameters)

# Now lets log the model and the accuracy
date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
with mlflow.start_run(run_name=f"mlflow-training-{date}"):
    
    # Hyperparameters are logged as parameters
    for hyperparameter, value in hyperparameters.items():
        mlflow.log_param(hyperparameter, value)
    
    # Accuracy is logged as a metric 
    mlflow.log_metric("accuracy", accuracy)
    
    # Model is logged as an artifact
    mlflow.sklearn.log_model(model, "model", signature=signature)

🏃 View run mlflow-training-2025-03-05-18-03-59 at: http://localhost:5001/#/experiments/301193664384281066/runs/73e099332d0f45ff9180439ff6354291
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


## 5. Run a hyperparameter search and log the results

In [5]:

import optuna
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


X, y = get_data()

def optimize_rf(trial):
    
    with mlflow.start_run(run_name=f"optuna-hp-{trial.number}-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"):
        # Set the hyperparameter values that we want to optimize
        n_estimators = trial.suggest_int('n_estimators', 1, 100)
        max_depth = trial.suggest_int('max_depth', 2, 10)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
        max_features = trial.suggest_float('max_features', 0.1, 1.0)
        
        
        # Create a random forest classifier using the suggested hyperparameters
        rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features)
        
        # Use cross-validation to evaluate the performance of the classifier
        scores = cross_val_score(rf, X, y, cv=5)
        
        # Log the hyperparameters and cross-validation scores to MLflow
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)
        mlflow.log_param('min_samples_split', min_samples_split)
        mlflow.log_param('min_samples_leaf', min_samples_leaf)
        mlflow.log_param('max_features', max_features)
        mlflow.log_metric('mean_cv_score', scores.mean())
    
    # Return the mean of the cross-validation scores as the objective value
    return scores.mean()

# Create an Optuna study
study = optuna.create_study()

# Run the optimization loop
study.optimize(optimize_rf, n_trials=50)

# Get the best hyperparameter values
best_params = study.best_params

c:\Users\Usuario\Documents\Master IA\MIA 24-25\EDEM_MDA2425\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-05 18:05:50,141] A new study created in memory with name: no-name-60a82e51-7025-4eb8-82db-097e77491131
[I 2025-03-05 18:05:51,081] Trial 0 finished with value: 0.9666666666666668 and parameters: {'n_estimators': 94, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.7880681117631002}. Best is trial 0 with value: 0.9666666666666668.


🏃 View run optuna-hp-0-2025-03-05-18-05-50 at: http://localhost:5001/#/experiments/301193664384281066/runs/d0daf1673cd94bf791838ed50b393481
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


[I 2025-03-05 18:05:51,317] Trial 1 finished with value: 0.96 and parameters: {'n_estimators': 8, 'max_depth': 8, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': 0.32029294144090137}. Best is trial 1 with value: 0.96.


🏃 View run optuna-hp-1-2025-03-05-18-05-51 at: http://localhost:5001/#/experiments/301193664384281066/runs/716566c59b1f4012830a8b3787ff9692
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


[I 2025-03-05 18:05:51,964] Trial 2 finished with value: 0.9533333333333334 and parameters: {'n_estimators': 83, 'max_depth': 9, 'min_samples_split': 20, 'min_samples_leaf': 20, 'max_features': 0.8168561374199689}. Best is trial 2 with value: 0.9533333333333334.


...

🏃 View run optuna-hp-47-2025-03-05-18-06-05 at: http://localhost:5001/#/experiments/301193664384281066/runs/5592d2ce506c4d46b0410362618df379
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


[I 2025-03-05 18:06:06,201] Trial 48 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 39, 'max_depth': 7, 'min_samples_split': 8, 'min_samples_leaf': 18, 'max_features': 0.23674401523831126}. Best is trial 18 with value: 0.7533333333333334.


🏃 View run optuna-hp-48-2025-03-05-18-06-05 at: http://localhost:5001/#/experiments/301193664384281066/runs/244a098f1453421e95a9f0e5bda15be2
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


[I 2025-03-05 18:06:06,526] Trial 49 finished with value: 0.9466666666666665 and parameters: {'n_estimators': 28, 'max_depth': 5, 'min_samples_split': 16, 'min_samples_leaf': 20, 'max_features': 0.5326839799821039}. Best is trial 18 with value: 0.7533333333333334.


🏃 View run optuna-hp-49-2025-03-05-18-06-06 at: http://localhost:5001/#/experiments/301193664384281066/runs/584881a446854b629bd59df5fba58e92
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


## 6. Register a model version using the best hyperparameters

In [6]:
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name=f"optuna-hp-final"):
    # Create the final model using the best hyperparameters
    final_model = RandomForestClassifier(**best_params)

    # Train the final model on the entire dataset
    final_model.fit(X, y)

    # Infer the model signature
    signature = infer_signature(X, final_model.predict(X))

    # Log the model to the "Models" section
    mlflow.sklearn.log_model(final_model, "random_forest_model", registered_model_name="random_forest_model", signature=signature)

Successfully registered model 'random_forest_model'.
2025/03/05 18:10:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 1


🏃 View run optuna-hp-final at: http://localhost:5001/#/experiments/301193664384281066/runs/f38d87102bcf479db22159759b3352c0
🧪 View experiment at: http://localhost:5001/#/experiments/301193664384281066


Created version '1' of model 'random_forest_model'.


## 7. Retrieve a model version and use it for prediction

In [10]:
import mlflow.pyfunc

model_name = "random_forest_model"
model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

model.predict(X[:5])

array([0, 0, 0, 0, 0])